In [2]:
from run_claude import run_claude
from swebench.harness.utils import load_swebench_dataset
import random
from evals import evaluate_results
import os
import sys
import subprocess
from pathlib import Path
import pandas as pd
import getpass

notebook_dir = Path().absolute()
sys.path.insert(0, str(notebook_dir.parent.parent))
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

sys.path.insert(0, str(notebook_dir.parent))
from constants import CLAUDE_CODE_PROMPT

2025-11-06 17:03:49,673 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/priyanjindal/.phoenix
2025-11-06 17:03:49,689 - phoenix.inferences.inferences - INFO - Dataset: phoenix_inferences_45ac80ad-8744-43c1-aeaf-36ea348feeb7 initialized


In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY") or getpass.getpass("Enter your Anthropic API key: ")
os.environ["PHOENIX_API_KEY"] = os.getenv("PHOENIX_API_KEY") or getpass.getpass("Enter your Phoenix API key: ")
HOSTNAME = os.getenv("PHOENIX_HOSTNAME") or getpass.getpass("Enter your Phoenix hostname: ")

In [19]:
LOOPS = 2
TRAIN_SIZE = 30
TEST_SIZE = 30
WORKERS = 50

In [11]:
dataset_name = "SWE-bench/SWE-bench_Lite"
dataset = load_swebench_dataset(dataset_name, "test")
print(len(dataset))

300


In [20]:
import random

dataset_name = "SWE-bench/SWE-bench_Lite"
dataset = load_swebench_dataset(dataset_name, "test")

# Define train and test repos
# By using certain repos for train and certain repos for test, we prevent overfitting to certain repositories
train_repos = {
    'django/django',
    'pytest-dev/pytest',
    'sphinx-doc/sphinx',
    'astropy/astropy',
    'psf/requests',
    'pylint-dev/pylint'
}

test_repos = {
    'sympy/sympy',
    'matplotlib/matplotlib',
    'scikit-learn/scikit-learn',
    'pydata/xarray',
    'mwaskom/seaborn',
    'pallets/flask'
}

# Split by repository
train_examples = [inst for inst in dataset if inst['repo'] in train_repos]
test_examples = [inst for inst in dataset if inst['repo'] in test_repos]
train_dataset = random.sample(train_examples, TRAIN_SIZE)
test_dataset = random.sample(test_examples, TEST_SIZE)

# Extract IDs for convenience
train_ids = [inst["instance_id"] for inst in train_dataset]
test_ids = [inst["instance_id"] for inst in test_dataset]

# Create DataFrames
train_pd = pd.DataFrame(train_dataset)
test_pd = pd.DataFrame(test_dataset)

# Verify the split
print(f"Train: {len(train_dataset)} instances ({len(train_dataset)/len(dataset)*100:.1f}%)")
print(f"Test: {len(test_dataset)} instances ({len(test_dataset)/len(dataset)*100:.1f}%)")
print(f"\nTrain repos: {sorted(train_repos)}")
print(f"Test repos: {sorted(test_repos)}")
print(f"\nTrain repo distribution:")
print(train_pd['repo'].value_counts())
print(f"\nTest repo distribution:")
print(test_pd['repo'].value_counts())

Train: 30 instances (10.0%)
Test: 30 instances (10.0%)

Train repos: ['astropy/astropy', 'django/django', 'psf/requests', 'pylint-dev/pylint', 'pytest-dev/pytest', 'sphinx-doc/sphinx']
Test repos: ['matplotlib/matplotlib', 'mwaskom/seaborn', 'pallets/flask', 'pydata/xarray', 'scikit-learn/scikit-learn', 'sympy/sympy']

Train repo distribution:
repo
django/django        21
pytest-dev/pytest     3
pylint-dev/pylint     2
sphinx-doc/sphinx     2
psf/requests          2
Name: count, dtype: int64

Test repo distribution:
repo
sympy/sympy                  18
matplotlib/matplotlib         6
scikit-learn/scikit-learn     4
pydata/xarray                 1
mwaskom/seaborn               1
Name: count, dtype: int64


In [21]:
from phoenix.client import Client

phoenix_client = Client(base_url=HOSTNAME, api_key=os.getenv("PHOENIX_API_KEY"))

train_dataset = phoenix_client.datasets.create_dataset(
    name="Claude Code: SWE-bench Train (30)",
    dataset_description="Claude Code: SWE-bench Train",
    dataframe=train_pd,
    input_keys=['problem_statement'],
    metadata_keys=['instance_id', 'test_patch'],
    output_keys=[]
)

test_dataset = phoenix_client.datasets.create_dataset(
    name="Claude Code: SWE-bench Test (30)",
    dataset_description="Claude Code: SWE-bench Test",
    dataframe=test_pd,
    input_keys=['problem_statement'],
    metadata_keys=['instance_id', 'test_patch'],
    output_keys=[]
)

2025-11-06 17:25:32,433 - phoenix.client.resources.datasets - INFO - Uploading dataset...
2025-11-06 17:25:32,526 - phoenix.client.resources.datasets - INFO - Dataset uploaded successfully. ID: RGF0YXNldDoyMg==, Version: RGF0YXNldFZlcnNpb246MjI=
2025-11-06 17:25:32,531 - phoenix.client.resources.datasets - INFO - Uploading dataset...
2025-11-06 17:25:32,564 - phoenix.client.resources.datasets - INFO - Dataset uploaded successfully. ID: RGF0YXNldDoyMw==, Version: RGF0YXNldFZlcnNpb246MjM=


In [16]:
from phoenix_experiments import log_experiment_to_phoenix

In [23]:
train_ids

['django__django-16408',
 'django__django-14016',
 'django__django-16595',
 'django__django-13448',
 'django__django-11815',
 'pytest-dev__pytest-8906',
 'pylint-dev__pylint-6506',
 'sphinx-doc__sphinx-8435',
 'django__django-11099',
 'pytest-dev__pytest-7168',
 'django__django-17087',
 'django__django-16400',
 'django__django-13447',
 'django__django-14411',
 'django__django-16229',
 'pylint-dev__pylint-7080',
 'django__django-14752',
 'pytest-dev__pytest-7220',
 'sphinx-doc__sphinx-10325',
 'django__django-11630',
 'psf__requests-863',
 'django__django-11999',
 'django__django-13158',
 'psf__requests-2317',
 'django__django-11848',
 'django__django-14608',
 'django__django-16873',
 'django__django-16379',
 'django__django-14787',
 'django__django-11905']

In [ ]:
train_df = run_claude(dataset_name=dataset_name, instance_ids=train_ids[:5], run_id="train_claude_0", ruleset=" ", workers=WORKERS)


Running Claude Code on 5 instances with 50 workers...
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_0/django__django-11815
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_1/django__django-13448
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_4/django__django-16595
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_3/django__django-16408
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_2/django__django-14016
[DEBUG] export_patch: ws=/Users/priyanjindal/materialized_repos/claude_0/django__django-11815
[DEBUG] staging changes for diff (excluding sqlite/db artifacts)
[DEBUG] export_patch: ws=/Users/priyanjindal/materialized_repos/claude_1/django__django-13448
[DEBUG] staging changes for diff (excluding sqlite/db artifacts)
[DEBUG] staged files:

[DEBUG] unstaged files:

[DEBUG] diff bytes=0
[DEBUG] wrote predictions: /var/folders/_w/glgvmwgs3s5

<frozen runpy>:128: RuntimeWarning: 'swebench.harness.run_evaluation' found in sys.modules after import of package 'swebench.harness', but prior to execution of 'swebench.harness.run_evaluation'; this may result in unpredictable behaviour


No instances to run.
Cleaning cached images...
Removed 0 images.
Total instances: 5
Instances submitted: 5
Instances completed: 0
Instances incomplete: 0
Instances resolved: 0
Instances unresolved: 0
Instances with empty patches: 5
Instances with errors: 0
Unstopped containers: 0
Unremoved images: 5
Report written to claude-code.train_claude_-1.json

Results Summary:
  Total instances: 5
  Passed: 0
  Failed: 5
  Pass rate: 0.0%

  Timeouts: 5
    ⏱ django__django-11815 (patch: 0 chars)
    ⏱ django__django-13448 (patch: 0 chars)
    ⏱ django__django-16595 (patch: 0 chars)
    ⏱ django__django-16408 (patch: 0 chars)
    ⏱ django__django-14016 (patch: 0 chars)

  Empty patches: 5
    ∅ django__django-16408 (timeout)
    ∅ django__django-14016 (timeout)
    ∅ django__django-16595 (timeout)
    ∅ django__django-13448 (timeout)
    ∅ django__django-11815 (timeout)



In [22]:
ruleset = " "

for loop in range(1):
    print(f"Running for loop: {loop}")

    train_run_id = f"train_{loop}"
    test_run_id = f"test_{loop}"

    train_df = run_claude(dataset_name=dataset_name, instance_ids=train_ids, run_id=train_run_id, ruleset=ruleset, workers=WORKERS)
    test_df = run_claude(dataset_name=dataset_name, instance_ids=test_ids, run_id=test_run_id, ruleset=ruleset, workers=WORKERS)

    test_df.to_csv(f"claude_code_results/test_results_{loop}.csv", index=False)
    
    train_acc = sum(train_df["pass_or_fail"] == "pass") / len(train_df)
    test_acc = sum(test_df["pass_or_fail"] == "pass") / len(test_df)
    print(f"Train Accuracy: {train_acc}")
    print(f"Test Accuracy: {test_acc}")

    # make sure any swebench package installations did not affect phoenix package

    subprocess.run([
        "/opt/anaconda3/envs/cline/bin/python3",
        "-m",
        "pip",
        "install",
        "-qq",
        "--upgrade",
        "arize-phoenix",
        "wrapt",
    ],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL)

    evaluated_train_results = evaluate_results(train_df)
    evaluated_train_results.to_csv(f"act_results/train_results_{loop}.csv", index=False)
    
    # Log experiment to Phoenix using REST API
    log_experiment_to_phoenix(
        hostname=HOSTNAME,
        api_key=os.getenv("PHOENIX_API_KEY"),
        dataset_obj=train_dataset,
        experiment_name=f"Train {loop}",
        experiment_df=evaluated_train_results,
        metadata={
            "loop": loop,
            "train_accuracy": train_acc,
            "test_accuracy": test_acc,
        }
    )

    # pl_optimizer = PromptLearningOptimizer(
    #     prompt=CLAUDE_CODE_PROMPT,
    #     model_choice="gpt-5",
    #     openai_api_key=os.getenv("OPENAI_API_KEY")
    # )
    # ruleset = pl_optimizer.optimize(
    #     dataset=evaluated_train_results,
    #     output_column="claude_patch",
    #     feedback_columns=["correctness", "explanation"],
    #     ruleset=ruleset,
    #     context_size_k=400000
    # )
    # with open(f"act_rulesets/ruleset_{loop}.txt", "w") as f:
    #     f.write(f"train_accuracy: {train_acc} \n")
    #     f.write(f"test_accuracy: {test_acc} \n")
    #     f.write(f"optimized ruleset_{loop}: \n {ruleset} \n")



Running for loop: 0
Running Claude Code on 30 instances with 50 workers...
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_21/psf__requests-2317
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_22/psf__requests-863
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_25/pytest-dev__pytest-7168
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_27/pytest-dev__pytest-8906
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_26/pytest-dev__pytest-7220
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_29/sphinx-doc__sphinx-8435
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_28/sphinx-doc__sphinx-10325
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_23/pylint-dev__pylint-6506
[DEBUG] ensure_git_baseline: ws=/Users/priyanjindal/materialized_repos/claude_24/pylint-dev__py

<frozen runpy>:128: RuntimeWarning: 'swebench.harness.run_evaluation' found in sys.modules after import of package 'swebench.harness', but prior to execution of 'swebench.harness.run_evaluation'; this may result in unpredictable behaviour
Traceback (most recent call last):
  File "/opt/anaconda3/envs/cline/lib/python3.11/site-packages/huggingface_hub/utils/_http.py", line 407, in hf_raise_for_status
    response.raise_for_status()
  File "/opt/anaconda3/envs/cline/lib/python3.11/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/datasets/SWE-bench/SWE-bench_Lite/resolve/69611d31007e1c6731db8bd5b5c3f2d33f5bab6e/SWE-bench_Lite.py

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/cline/lib/python3.11/site-packages/datasets/load.py", line 982, in datase

KeyboardInterrupt: 